In [ ]:
import re
import boto3
from datetime import datetime, timedelta

In [ ]:
def list_replication_instances():
    client = boto3.client('dms', region_name="sa-east-1")
    response = client.describe_replication_configs()
    replication_instances = response.get('ReplicationConfigs', [])

    results = []
    for instance in replication_instances:
        results.append({"name": instance["ReplicationConfigIdentifier"], "arn": instance["ReplicationConfigArn"]})

    return results

In [ ]:
def normalize_date(date):
    if date is None:
        return date
    if not isinstance(date, str):
        return date - timedelta(hours=3)
    return datetime.strptime(date, "%Y-%m-%d %H:%M:%S.%f") - timedelta(hours=3)

In [ ]:
def check_value_func(statistics):
    value = 0
    if statistics.get("AppliedInserts") != 0:
        value = statistics.get("AppliedInserts")
    elif statistics.get("AppliedDeletes") != 0:
        value = statistics.get("AppliedDeletes")
    elif statistics.get("AppliedUpdates") != 0:
        value = statistics.get("AppliedUpdates")
    return value

In [ ]:
def describe_table_statistics(replication_task_arn, prefix):
    results = []

    client = boto3.client('dms', region_name="sa-east-1")
    response = client.describe_replication_table_statistics(ReplicationConfigArn=replication_task_arn, MaxRecords=20)
    table_statistics = response.get('ReplicationTableStatistics', [])

    for statistics in table_statistics:
        table_name = f"{prefix}_{statistics.get('TableName')}"
        table_name = table_name.replace("_default", "")
        table_name = re.sub(r"_\d{1,4}.*", "", table_name)
        exists = any(table_name in d for d in results)
        if not exists:
            results.append({
                "table_name": table_name,
                "last_update_time": str(normalize_date(statistics.get("LastUpdateTime"))),
                "table_state": statistics.get('TableState'),
                "rows_inserted": check_value_func(statistics)
            })
    
    while True:
        if "Marker" in response:
            response = client.describe_replication_table_statistics(ReplicationConfigArn=replication_task_arn, MaxRecords=20, Marker=response["Marker"])
            table_statistics = response.get('ReplicationTableStatistics', [])
            
            for statistics in table_statistics:
                table_name = f"{prefix}_{statistics.get('TableName')}"
                table_name = table_name.replace("_default", "")
                table_name = re.sub(r"_\d{1,4}.*", "", table_name)
                exists = any(table_name in d for d in results)
                if not exists:
                    results.append({
                        "table_name": table_name,
                        "last_update_time": str(normalize_date(statistics.get("LastUpdateTime"))),
                        "table_state": statistics.get('TableState'),
                        "rows_inserted": check_value_func(statistics)
                    })
        else:
            break

    return results

In [ ]:
map_db = {
    "db1-cdc": "db1",
    "db2-cdc": "db2",
    # "db3-cdc": "db3"
}

all_tables = {}
results = list_replication_instances()

for result in results:
    if result["name"] in map_db.keys():
        db = map_db[result["name"]]
        all_tables[db] = describe_table_statistics(result["arn"], db)

dbutils.jobs.taskValues.set(key="db1_all_tables", value=all_tables["db1"])
dbutils.jobs.taskValues.set(key="db2_all_tables", value=all_tables["db2"])